In [1]:
import pandas as pd
import os
import pyAgrum.lib.notebook as gnb 


ot_odr_filename = os.path.join("./data", "OT_ODR.csv.bz2")
ot_odr_df = pd.read_csv(ot_odr_filename,
                        compression="bz2",
                        sep=";")

equipements_filename = os.path.join("./data", 'EQUIPEMENTS.csv')
equipements_df = pd.read_csv(equipements_filename,
                             sep=";")

In [2]:
ot_odr_df

,OT_ID,ODR_ID,ODR_LIBELLE,TYPE_TRAVAIL,DUREE_TRAVAIL,SYSTEM_N1,SYSTEM_N2,SYSTEM_N3,EQU_ID,DATE_OT,KILOMETRAGE,SIG_ORGANE,SIG_CONTEXTE,SIG_OBS,LIGNE
0,OT000000000,OM000000000,REMPLACEMENT D'UNE GLACE LAT VOYAGEUR,CARROSSERIE,4.00,EQUIPEMENT DE CARROSSERIE,VITRAGE,VITRAGE LAT,E00005934,2011-03-29 19:26:06,149698.557783,GLACE/BAIE,INTERIEUR/GAUCHE/ARRIERE,DEBOITE,L0482
1,OT000000001,OM000000001,REMPLACEMENT D'UN COMMODO DE SIGNALISATION,ELECTRICITE,0.50,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00004713,2011-05-03 20:01:31,225035.016000,KLAXON/GONG,AVANT,ABSENT,L0147
2,OT000000002,OM000000002,REMPLACEMENT D'UN CARDAN DE LIAISON SUR CREMAI...,MECANIQUE,1.50,EQUIPEMENT CHASSIS,EQUIPEMENT DE DIRECTION,COMMANDE DE DIRECTION,E00006037,2011-05-05 14:40:22,71148.834963,VOITURE,A L'ACCELERATION,VIBRE,L0368
3,OT000000003,OM000000003,REMPLACEMENT D'UN PARE-CHOCS AVG,CARROSSERIE,0.50,EQUIPEMENT DE CARROSSERIE,ELEMENT CARROSSERIE EXT,PROTECTION AV,E00005670,2011-05-07 07:43:27,116441.657358,PARE-CHOCS,AVANT/GAUCHE,CASSE,L0066
4,OT000000004,OM000000004,REMPLACEMENT D'UN POTENTIOMETRE DE PORTE NUMERO 1,ELECTRICITE,0.50,EQUIPEMENT DE CARROSSERIE,PORTE,COMMANDE PORTE,E00004009,2011-05-18 10:56:50,0.000000,SECURITE PORTE,ARRIERE,BLOQUE,L0247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506553,OT000405952,OM000506539,REMPLACEMENT D'UN FEU DE RECUL,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00040793,2019-09-30 20:38:09,296005.373397,ECLAIRAGE FEUX EXTERIEURS,ARRIERE/EXTERIEUR/DROIT,CASSE,L0283
506554,OT000405953,OM000506546,REMPLACEMENT D'UN ECLAIRAGE EXT,MECANIQUE,0.75,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00274690,2019-09-30 21:21:20,153050.080050,ECLAIRAGE FEUX EXTERIEURS,ARRIERE/LATERAL,CASSE,L0116
506555,OT000405954,OM000506536,REMPLACEMENT D'UN PASSE SANS CONTACT,EQUIPEMENT EMBARQUE,0.03,EQUIPEMENT EMBARQUE,TELEBILLETIQUE,PASSE SANS CONTACT,E00256452,2019-09-30 21:39:29,175063.182439,AVTT,AVANT/PORTE,INTERMITTENT,L0134
506556,OT000405955,OM000506545,REMPLACEMENT D'UNE LAMPE DE FEU DE GABARIT,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00006122,2019-09-30 21:55:28,437053.614263,ECLAIRAGE FEUX EXTERIEURS,LATERAL/HAUT/GAUCHE,NE FONCTIONNE PAS,L0270


In [3]:
equipements_df

,EQU_ID,MODELE,CONSTRUCTEUR,MOTEUR
0,E00005610,MD040,C008,MT035
1,E00006320,MD017,C007,MT014
2,E00005098,MD037,C007,MT010
3,E00000321,MD017,C007,MT014
4,E00022350,MD014,C007,MT013
...,...,...,...,...
4882,E00271009,MD023,C007,MT021
4883,E00341892,MD063,C007,MT023
4884,E00331551,MD005,C000,MT040
4885,E00373890,MD021,C011,MT0-1


In [4]:
# Fusion des DataFrames sur la colonne 'EQU_ID'
df_merged = pd.merge(ot_odr_df, equipements_df, on='EQU_ID')

# Affichage du DataFrame fusionné
df_merged

,OT_ID,ODR_ID,ODR_LIBELLE,TYPE_TRAVAIL,DUREE_TRAVAIL,SYSTEM_N1,SYSTEM_N2,SYSTEM_N3,EQU_ID,DATE_OT,KILOMETRAGE,SIG_ORGANE,SIG_CONTEXTE,SIG_OBS,LIGNE,MODELE,CONSTRUCTEUR,MOTEUR
0,OT000000000,OM000000000,REMPLACEMENT D'UNE GLACE LAT VOYAGEUR,CARROSSERIE,4.00,EQUIPEMENT DE CARROSSERIE,VITRAGE,VITRAGE LAT,E00005934,2011-03-29 19:26:06,149698.557783,GLACE/BAIE,INTERIEUR/GAUCHE/ARRIERE,DEBOITE,L0482,MD017,C007,MT014
1,OT000003071,OM000003274,REMPLACEMENT D'UN POTENTIOMETRE DE PORTE NUMERO 3,MECANIQUE,0.50,EQUIPEMENT DE CARROSSERIE,PORTE,SECURITES PORTES,E00005934,2012-09-29 21:30:27,212886.557303,PORTE,ARRIERE,FONCTIONNE MAL,L0135,MD017,C007,MT014
2,OT000003071,OM000003275,REMPLACEMENT D'UN POTENTIOMETRE DE PORTE NUMERO 4,MECANIQUE,0.50,EQUIPEMENT DE CARROSSERIE,PORTE,SECURITES PORTES,E00005934,2012-09-29 21:30:27,212886.557303,PORTE,ARRIERE,FONCTIONNE MAL,L0135,MD017,C007,MT014
3,OT000003071,OM000003276,REMPLACEMENT D'UN INTERRUPTEUR DE COMMANDE DE ...,MECANIQUE,0.08,EQUIPEMENT DE CARROSSERIE,PORTE,COMMANDE PORTE,E00005934,2012-09-29 21:30:27,212886.557303,PORTE,ARRIERE,FONCTIONNE MAL,L0135,MD017,C007,MT014
4,OT000005214,OM000005645,REMPLACEMENT DU CIRCUIT IMPRIME GAUCHE DU TABL...,MECANIQUE,0.20,EQUIPEMENT ELECTRIQUE,CIRCUIT DE GESTION ELECTRIQUE,CONNEXION - CABLAGE,E00005934,2013-02-11 21:45:40,228132.331835,DEMARRAGE,AU DEMARRAGE/POSTE CONDUITE,FAUX CONTACT,L0066,MD017,C007,MT014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506553,OT000403521,OM000503032,REMPLACEMENT D'UN FEU DE GABARIT,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00364746,2019-09-17 00:22:38,1605.000000,ECLAIRAGE FEUX EXTERIEURS,LATERAL/DROIT,CASSE,L0098,MD064,C007,MT024
506554,OT000404355,OM000504846,REMPLACEMENT D'UN CABLAGE PASSE SANS CONTACT,ELECTRICITE,0.25,EQUIPEMENT EMBARQUE,TELEBILLETIQUE,PASSE SANS CONTACT,E00362393,2019-09-20 20:30:07,5145.078211,AVTT,GAUCHE,NE FONCTIONNE PAS,L0077,MD032,C006,MT016
506555,OT000405069,OM000505089,REMPLACEMENT D'UNE LAMPE DE CODE / PHARE,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00364751,2019-09-24 23:31:41,9453.675111,ECLAIRAGE FEUX EXTERIEURS,GAUCHE/DROIT/AVANT,DESSERRE,L0098,MD064,C007,MT024
506556,OT000405447,OM000505789,REMPLACEMENT D'UN PASSE SANS CONTACT,EQUIPEMENT EMBARQUE,0.06,EQUIPEMENT EMBARQUE,TELEBILLETIQUE,PASSE SANS CONTACT,E00370753,2019-09-26 19:52:05,2599.051019,AVTT,EN MONTEE,VOYANT HS,L0178,MD021,C011,MT051


In [5]:
df_merged['ODR_LIBELLE'].nunique()


1410

In [6]:
df_merged['SIG_ORGANE'].nunique()


116

In [7]:
df_merged['SIG_CONTEXTE'].nunique()


5509

In [8]:
df_merged['SIG_OBS'].nunique()

59

In [9]:
import pyAgrum as gum # La librairie pyAgrum

# Vérification des versions
{
    "pyagrum": gum.__version__, 
}

{'pyagrum': '1.8.0'}

In [10]:
bn = gum.BayesNet("rn")
bn

(pyAgrum.BayesNet<double>@000001C225A5F3E0) BN{nodes: 0, arcs: 0, domainSize: 1, dim: 0, mem: 0o}

In [11]:
va_Moteur = gum.LabelizedVariable("MOTEUR", "Moteur", df_merged['MOTEUR'].nunique())
va_Moteur

(pyAgrum.LabelizedVariable@000001C2250C1F90) MOTEUR:Labelized({0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|28|29|30|31|32|33|34|35|36|37|38|39|40|41|42|43|44|45|46|47|48|49|50|51|52})

In [12]:
va_C = gum.LabelizedVariable("CONSTRUCTEUR", "Constructeur", df_merged['CONSTRUCTEUR'].nunique())
va_Model = gum.LabelizedVariable("MODELE", "Model", df_merged['MODELE'].nunique())
va_SIG_OBS = gum.LabelizedVariable("SIG_OBS", "SIG_OBS", df_merged['SIG_OBS'].nunique())
va_SYS_N1 = gum.LabelizedVariable("SYSTEM_N1", "SYSTEM_N1 : chute observée ?", df_merged['SYSTEM_N1'].nunique())

In [13]:
bn

(pyAgrum.BayesNet<double>@000001C225A5F3E0) BN{nodes: 0, arcs: 0, domainSize: 1, dim: 0, mem: 0o}

In [14]:
for va in [va_Moteur,va_C,va_Model,va_SIG_OBS,va_SYS_N1]:
    bn.add(va)
bn

(pyAgrum.BayesNet<double>@000001C225A5F3E0) BN{nodes: 5, arcs: 0, domainSize: 10^7.3944, dim: 191, mem: 1Ko 544o}

In [15]:
bn.addArc("MOTEUR", "SYSTEM_N1")
bn.addArc("CONSTRUCTEUR", "SYSTEM_N1")
bn.addArc("MODELE", "SYSTEM_N1")
bn.addArc("SIG_OBS", "SYSTEM_N1")
bn

(pyAgrum.BayesNet<double>@000001C225A5F3E0) BN{nodes: 5, arcs: 4, domainSize: 10^7.3944, dim: 22317581, mem: 189Mo 192Ko 896o}